# Evaluating Model

In [2]:
%%sh

pip3 install -r requirements.txt --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torcheia 1.0.0 requires torch==1.5.1, but you have torch 1.13.1 which is incompatible.
sagemaker 2.127.0 requires boto3<2.0,>=1.26.28, but you have boto3 1.17.106 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.44 requires botocore==1.29.44, but you have botocore 1.20.106 which is incompatible.
awscli 1.27.44 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.44 requires s3transfer<0.7.0,>=0.6.0, but you have s3transfer 0.4.2 which is incompatible.


In [5]:
from typing import Any, Dict, Optional, Tuple

import os
import subprocess
import torch
import json
import tarfile

import pytorch_lightning as pl
import torchvision.transforms as T
import torch.nn.functional as F

from pathlib import Path
from torchvision.datasets import ImageFolder
from pytorch_lightning.plugins.environments import LightningEnvironment
from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional import accuracy
from pytorch_lightning import loggers as pl_loggers
from datetime import datetime

from scripts.model import LitResnet
from scripts.dataset import IntelCapstoneDataModule

In [12]:
def eval_model(trainer, model, datamodule):
    test_res = trainer.test(model, datamodule)[0]
    
    report_dict = {
        "multiclass_classification_metrics": {
            "accuracy": {
                "value": test_res["test/acc"],
                "standard_deviation": "0",
            },
        },
    }
    
    eval_folder = Path("tmp")
    
    out_path = eval_folder / "evaluation.json"
    
    print(f":: Writing to {out_path.absolute()}")
    
    with out_path.open("w") as f:
        f.write(json.dumps(report_dict))
        

In [13]:

model_path = "tmp/model.tar.gz"
with tarfile.open(model_path) as tar:
    tar.extractall(path=".")

datamodule = IntelCapstoneDataModule(
    train_data_dir='test/',
    test_data_dir='test/',
    num_workers=os.cpu_count(),
    albumentations = '{}'
)
datamodule.setup()

model = LitResnet.load_from_checkpoint(checkpoint_path="last.ckpt")

trainer = pl.Trainer(
    accelerator="auto",
)

print(":: Evaluating Model")
eval_model(trainer, model, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


:: Evaluating Model


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.7053298950195312     │
│         test/loss         │    0.7563663721084595     │
└───────────────────────────┴───────────────────────────┘

:: Writing to /home/ec2-user/SageMaker/sagemaker-pipeline/capstone/tmp/evaluation.json
